# Install and import

In [1]:
!pip install --upgrade "ibm-watson>=4.5.0"

     |████████████████████████████████| 385 kB 3.5 MB/s eta 0:00:01
     |████████████████████████████████| 198 kB 9.0 MB/s eta 0:00:01
  Created wheel for ibm-watson: filename=ibm_watson-4.7.1-py3-none-any.whl size=379130 sha256=077edee8c50b8259b4975b5e6b01e3194713d6a5421397e7fe5f341b82412da8
  Stored in directory: /home/jovyan/.cache/pip/wheels/63/0c/16/b2af79aace901934f32f1503a95b338c7bdca6ab2cfa0881e6
  Created wheel for ibm-cloud-sdk-core: filename=ibm_cloud_sdk_core-1.7.3-py3-none-any.whl size=45943 sha256=1e266fcb815d925610fcfdbbfe3269b0f6eb0ce7c5caa81734b01481b45ca6b7
  Stored in directory: /home/jovyan/.cache/pip/wheels/c6/5f/fb/c2daef3706ded7e718641a60d097be3116a6b70a8f236753f2
Successfully built ibm-watson ibm-cloud-sdk-core
You should consider upgrading via the '/opt/venv/bin/python -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
import csv
from ibm_watson import ToneAnalyzerV3
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
import numpy as np
import time
from google.colab import files

# Load the data

In [3]:
data = pd.read_csv('ug_processed.csv')

In [4]:
qualitative_data = data.loc[:,['preferred_mode','why_mode', 'preference', 'why_preference', 'remote_changes', 'prior_changes']]
qualitative_data['mode_tone'] = np.zeros((data.shape[0], 1))
qualitative_data['preference_tone'] = np.zeros((data.shape[0], 1))
qualitative_data['remote_changes_tone'] = np.zeros((data.shape[0], 1))
qualitative_data['prior_changes_tone'] = np.zeros((data.shape[0], 1))

In [5]:
print(qualitative_data.shape)
qualitative_data.head()

(4789, 10)


,preferred_mode,why_mode,preference,why_preference,remote_changes,prior_changes,mode_tone,preference_tone,remote_changes_tone,prior_changes_tone
0,recorded,Easier to follow and look back on,In-person Courses,Easier to debate,More live debate,NaN,0.0,0.0,0.0,0.0
1,live,Most similar to normality and provides a routine,In-person Courses,It’s easier to learn and ask questions,Not sure,NaN,0.0,0.0,0.0,0.0
2,live,It is more interactive,In-person Courses,I find them more engaging,I would make them more interactive,NaN,0.0,0.0,0.0,0.0
3,live,Allows more of a conversation to flow and ques...,Online Courses,Gives more freedom,More group activities within the class time to...,Number of fun/group activities,0.0,0.0,0.0,0.0
4,live,NaN,In-person Courses,I simply enjoy the university life,The classes should be as regular as during a n...,NaN,0.0,0.0,0.0,0.0


# Build the model

In [ ]:
authenticator = IAMAuthenticator('MPBO7gcnQsN_fUW4RBCc3Gs2JV15sw5RqJLpTOC9zMwi')
tone_analyzer = ToneAnalyzerV3(
    version='2017-09-21',
    authenticator=authenticator
)

tone_analyzer.set_service_url('https://api.us-south.tone-analyzer.watson.cloud.ibm.com')

In [ ]:
def sentiment(data):
    if not pd.isna(data):
        json_output = tone_analyzer.tone(data, content_type='text/plain').result['document_tone']['tones']
        return json_output
    else:
        return np.NaN

In [ ]:
def sentiment_applied(data, col):
    start = time.time()
    sentiments = data[col].apply(sentiment)
    end = time.time()
    return end-start, sentiments

# Execute the model

In [ ]:
sentiment(qualitative_data['why_preference'].iloc[-2])

[{'score': 0.880435, 'tone_id': 'joy', 'tone_name': 'Joy'}]

In [ ]:
sentiment(qualitative_data['why_preference'].iloc[1])

[{'score': 0.730335, 'tone_id': 'analytical', 'tone_name': 'Analytical'}]

In [ ]:
duration1, sentiment_why_mode = sentiment_applied(qualitative_data, 'why_mode')

In [ ]:
sentiment_why_mode.to_csv('sentiment_why_mode.csv')
files.download('sentiment_why_mode.csv')

In [ ]:
duration2, sentiment_why_preference = sentiment_applied(qualitative_data, 'why_preference')
sentiment_why_preference.to_csv('sentiment_why_preference.csv')
files.download('sentiment_why_preference.csv')

In [ ]:
duration3, sentiment_remote_change = sentiment_applied(qualitative_data, 'remote_changes')
sentiment_remote_change.to_csv('sentiment_remote_change.csv')
files.download('sentiment_remote_change.csv')

In [ ]:
duration4, sentiment_prior_change = sentiment_applied(qualitative_data, 'prior_changes')
sentiment_prior_change.to_csv('sentiment_prior_change.csv')
files.download('sentiment_prior_change.csv')